In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# Parametry
batch_size = 64
learning_rate = 0.001
num_epochs = 2

# Transformacje danych
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Pobieranie danych
train_dataset = datasets.MNIST(root='data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='data', train=False, transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [7]:
class SimpleFCNNWithDropout(nn.Module):
    def __init__(self):
        super(SimpleFCNNWithDropout, self).__init__()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)
        
        # Dodanie Dropout
        self.dropout = nn.Dropout(p=0.5)  # Dropout z prawdopodobieństwem 50%
        
    def forward(self, x):
        x = x.view(-1, 28*28)  # spłaszczenie obrazu
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)  # Dropout po pierwszej warstwie
        x = torch.relu(self.fc2(x))
        x = self.dropout(x)  # Dropout po drugiej warstwie
        x = self.fc3(x)
        return x

# Inicjalizacja modelu i kryterium
model = SimpleFCNNWithDropout()
criterion = nn.CrossEntropyLoss()

In [9]:
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [11]:
def train(model, train_loader, criterion, optimizer, num_epochs):
    model.train()  # Ustawienie modelu w tryb treningowy
    for epoch in range(num_epochs):
        running_loss = 0
        for images, labels in train_loader:
            # Zero gradientów
            optimizer.zero_grad()
            
            # Przekazanie przez sieć
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            # Obliczenie gradientów i aktualizacja wag
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        avg_loss = running_loss / len(train_loader)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

train(model, train_loader, criterion, optimizer, num_epochs)

Epoch [1/2], Loss: 0.7668
Epoch [2/2], Loss: 0.4765


In [13]:
def evaluate(model, test_loader, criterion):
    model.eval()  # Ustawienie modelu w tryb ewaluacji (wyłącza Dropout)
    test_loss = 0
    correct = 0
    with torch.no_grad():  # Wyłącz gradienty podczas ewaluacji
        for images, labels in test_loader:
            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            
            # Przewidywanie
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
    
    test_loss /= len(test_loader)
    accuracy = correct / len(test_loader.dataset)
    print(f"Test Loss: {test_loss:.4f}, Accuracy: {accuracy * 100:.2f}%")

evaluate(model, test_loader, criterion)

Test Loss: 0.2334, Accuracy: 92.86%
